In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-point-forecast-valweights/__results__.html
/kaggle/input/m5-point-forecast-valweights/__notebook__.ipynb
/kaggle/input/m5-point-forecast-valweights/weights_df
/kaggle/input/m5-point-forecast-valweights/custom.css
/kaggle/input/m5-point-forecast-valweights/__output__.json
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv


In [2]:
import pickle, sys
from collections import defaultdict

In [3]:
sales_train_val = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')

In [4]:
sales_train_val


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [5]:
with open('../input/m5-point-forecast-valweights/weights_df', 'rb') as f:
    val_weights = pickle.load(f)

In [6]:
val_weights

,item_id,store_id,day_of_first_sale,weight,K,all_zeros,sales_mean,sales_std
0,HOBBIES_1_001,CA_1,902,0.000060,0.855926,0,0.592483,0.849799
1,HOBBIES_1_002,CA_1,144,0.000002,1.278055,0,0.278123,0.586817
2,HOBBIES_1_003,CA_1,1106,0.000013,1.128216,0,0.353160,0.691916
3,HOBBIES_1_004,CA_1,37,0.000063,0.372943,0,1.751599,1.994428
4,HOBBIES_1_005,CA_1,113,0.000029,0.611990,0,1.022778,1.300441
...,...,...,...,...,...,...,...,...
30485,FOODS_3_823,WI_3,3,0.000005,0.863262,0,0.534031,1.177291
30486,FOODS_3_824,WI_3,6,0.000006,0.981145,0,0.374934,0.813791
30487,FOODS_3_825,WI_3,2,0.000027,0.659478,0,0.892726,1.380701
30488,FOODS_3_826,WI_3,940,0.000009,0.677787,0,0.720452,1.249856


In [7]:
mask = (val_weights.weight != 0).values

In [8]:
val_weights_mask = val_weights.loc[mask]

In [9]:
val_weights_mask = val_weights_mask.reset_index(drop=True)

In [10]:
# item_store and all_id implicit
agg_levels = (
            'cat_id',
            'state_id',
            'dept_id',
            'store_id',
            'item_id',
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            ['item_id', 'state_id'],
            )

In [11]:
sales_train_val_mask = sales_train_val.loc[mask]

In [12]:
sales_train_val_mask = sales_train_val_mask.reset_index(drop=True)

In [13]:
agg_levels

('cat_id',
 'state_id',
 'dept_id',
 'store_id',
 'item_id',
 ['state_id', 'cat_id'],
 ['state_id', 'dept_id'],
 ['store_id', 'cat_id'],
 ['store_id', 'dept_id'],
 ['item_id', 'state_id'])

In [14]:
agg_groups = []
for ag in agg_levels:
    st_gp = sales_train_val_mask.groupby(by=ag)
    for _,_df in st_gp:
        idxs = _df.index.values
        ser = _df.iloc[:,6:].values.sum(axis=0)
        nz_i = np.sort(np.where(ser>0)[0])[0]
        K = ((ser[nz_i+1:] - ser[nz_i:-1])**2).mean()
        K = (1.0 / K)**0.5
        agg_groups.append((idxs,K))

In [15]:
len(agg_groups)

12139

In [16]:
all_ser = sales_train_val_mask.iloc[:,6:].values.sum(axis=1)
all_nz_i = np.sort(np.where(all_ser>0)[0])[0]
all_K = ((all_ser[all_nz_i+1:] - all_ser[all_nz_i:-1])**2).mean()
all_K = (1.0 / all_K)**0.5

In [17]:
all_idxs = sales_train_val_mask.index.values

In [18]:
agg_groups.append((all_idxs, all_K))

In [19]:
with open('agg_groups_idxs_K','wb') as f:
    pickle.dump(agg_groups, f)

In [20]:
! ls -alrh

total 3.5M
-rw-r--r-- 1 root root 3.5M Jun 22 15:33 agg_groups_idxs_K
---------- 1 root root 5.1K Jun 22 15:33 __notebook__.ipynb
drwxr-xr-x 6 root root 4.0K Jun 22 15:33 ..
drwxr-xr-x 2 root root 4.0K Jun 22 15:33 .
